In [1]:
from pyspark.sql.session import SparkSession

In [2]:
spark = SparkSession.builder.appName("logit").getOrCreate()

21/11/01 00:08:06 WARN Utils: Your hostname, xtian-pc resolves to a loopback address: 127.0.1.1; using 192.168.1.39 instead (on interface enp5s0)
21/11/01 00:08:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/11/01 00:08:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
from pyspark.ml.classification import LogisticRegression

In [4]:
my_data = spark.read.format("libsvm").load("sample_libsvm_data.txt")

21/11/01 00:08:45 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [5]:
my_data.show(5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
+-----+--------------------+
only showing top 5 rows



In [6]:
my_logreg_model = LogisticRegression()

In [12]:
lr_train, lr_test = my_data.randomSplit([0.7, 0.3])

In [13]:
fitted_logreg = my_logreg_model.fit(lr_train)

In [14]:
log_summary = fitted_logreg.summary

In [15]:
log_summary.predictions

DataFrame[label: double, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [10]:
log_summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[19.8534775947478...|[0.99999999761359...|       0.0|
|  1.0|(692,[158,159,160...|[-20.377398194908...|[1.41321555111051...|       1.0|
|  1.0|(692,[124,125,126...|[-27.401459284891...|[1.25804865126979...|       1.0|
|  1.0|(692,[152,153,154...|[-18.862741612668...|[6.42710509170301...|       1.0|
|  1.0|(692,[151,152,153...|[-20.483011833009...|[1.27157209200602...|       1.0|
|  0.0|(692,[129,130,131...|[19.8506078990277...|[0.99999999760673...|       0.0|
|  1.0|(692,[158,159,160...|[-20.337256674833...|[1.47109814695578...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.595579753418...|[3.08850168102648...|       1.0|
|  0.0|(692,[154,155,156...|[19.2708803215613...|[0.99999999572670...|       0.0|
|  0.0|(692,[127

In [16]:
preds_and_labes = fitted_logreg.evaluate(lr_test)

In [18]:
from pyspark.ml.evaluation import (BinaryClassificationEvaluator, 
                                MulticlassClassificationEvaluator)

In [19]:
my_eval = BinaryClassificationEvaluator()

In [20]:
final_roc = my_eval.evaluate(preds_and_labes.predictions)

In [21]:
final_roc

1.0